In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Redes Neuronales Artificiales

In [2]:
#Apertura de archivo csv a un dataframe de pandas

#revisar la ruta para tu caso particular
ruta = "c:\\Users\\jrgui\\Documents\\GitHub\\Machine_Learning\\Redes_Neuronales\\datasets\\Churn_Modelling.csv"
dataset = pd.read_csv(ruta)
dataset.head(5)

#Datos de los clintes de un banco, queremos entrenar una red que diga si el cliete se va o se queda

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocesado de datos
from sklearn import preprocessing

#Se escogen las variable dependiente e independientes
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values


#Transformar a variables categoricas ordinales
labelencoder_x_paises = preprocessing.LabelEncoder()
#cambio los nombre de los paises en categorias ordinales
X[:,1] = labelencoder_x_paises.fit_transform(X[:,1])
labelencoder_x_generos = preprocessing.LabelEncoder()
#cambio los generos en categorias ordinales
X[:,2] = labelencoder_x_generos.fit_transform(X[:,2])


#Ahorra se debe pasar a variable dummy con onehotencoder
#tranforma el arreglo en un arreglo de numpy
data = np.array(X)
# Separar la primera columna del resto
paises_column = data[:, 1].reshape(-1, 1)
generos_column = data[:, 2].reshape(-1, 1)

demas_columns = data[:, 3:]
primera_columns = data[:, 0].reshape(-1, 1)

# Crear el codificador OneHotEncoder
encoder_paises = preprocessing.OneHotEncoder(sparse_output=False)
encoder_generos = preprocessing.OneHotEncoder(sparse_output=False)

# Ajustar y transformar la primera columna
paises_column_encoded = encoder_paises.fit_transform(paises_column)
generos_column_encoded = encoder_paises.fit_transform(generos_column)

# Concatenar la columna codificada con las demás columnas
X = np.hstack((paises_column_encoded, primera_columns, generos_column_encoded, demas_columns))

#para evitar la colinealidad se deben eliminar 2 columnas una de paises y una de generos
X = np.delete(X, (0,4), axis=1)


#Division de Datos - entrenamiento y validacion
#herramienta para dividir los datos
from sklearn.model_selection import train_test_split
#divide los datos en 20% para la validacion y se colocar una semilla para hacer la division
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

#Estandarizacion
sc_X = preprocessing.StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [11]:
# Construir la Red Neuronal

import keras
from keras.models import Sequential
from keras.layers import Dense

#Inicializar la red neuronal
clasificador = Sequential()

#Capa de entrada y primera capa oculta
clasificador.add(Dense(units=6, kernel_initializer="uniform", activation="relu"))

#Segunda capa oculta
clasificador.add(Dense(units=6, kernel_initializer="uniform", activation="relu"))

#Capa de salida
clasificador.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

#Compilar la red
clasificador.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])



In [17]:
#Ajustar la Red Neuronal

clasificador.fit(X_train, y_train, batch_size=10, epochs=30)

Epoch 1/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8527 - loss: 0.3446
Epoch 2/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8729 - loss: 0.3138
Epoch 3/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8608 - loss: 0.3433
Epoch 4/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8627 - loss: 0.3346
Epoch 5/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8612 - loss: 0.3411
Epoch 6/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8638 - loss: 0.3376
Epoch 7/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8659 - loss: 0.3282
Epoch 8/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8652 - loss: 0.3369
Epoch 9/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8662 - loss: 0.3320
Epoch 10/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8608 - loss: 0.3373
Epoch 11/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8616 - loss: 0.3343
Epoch 12/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [18]:
#Prueba de las predicciones

#La salida son probabilidades
y_pred = clasificador.predict(X_test)
#para clasificar si va a abandonar el banco
y_pred = (y_pred > 0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [19]:
#matriz de confucion

#Comprobacion del resultado - Matriz de confucion
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

#Da un buen resultado la prediccion


array([[1510,   85],
       [ 196,  209]], dtype=int64)